In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

Cm = 1 # membrane capacitance
g_Na, g_K, g_L, = 120, 36, 0.3 # Maximam open condutance in each ion (mS/cm^2)
E_Na, E_K, E_L = 60, -88, -54.387 # Equilbrium potential in each ion (mV)

In [ ]:
# Transition rate (n = K activation parameter for hyperporalization)
def alpha_n(V): return 0.01*(V+55)/(1-np.exp(-(V+55)/10))
def beta_n(V): return 0.125*np.exp(-(V+65)/80)
# (m = Na activation parameter for depolarization)
def alpha_m(V): return 0.1*(V+40)/(1-np.exp(-(V+40)/10))
def beta_m(V): return 4*np.exp(-(V+65)/18)
# (h = Na inactivation parameter for refractory period)
def alpha_h(V): return 0.07*np.exp(-(V+65)/20)
def beta_h(V): return 1/(1+np.exp(-(V+35)/10))

In [ ]:
# voltage dependent membrane condutance
def g_NaV(V, m ,h): return g_Na * m**3*h
def g_KV(V, n): return g_K * n**4
def g_LV(V): return g_L

In [ ]:
# current (mA)
def I_Na(V, m, h): return g_NaV(V, m, h) * (V-E_Na)
def I_K(V, n): return g_KV(V, n) * (V- E_K)
def I_L(V): return g_LV(V) * (V-E_L)

In [ ]:
# input stimulus
t = np.arange(0, 350, 0.01)
def I_inj(t):
    return 10*(t>50)-10*(t>100)+40*(t>150)-40*(t>200)-5*(t>250)+5*(t>300)

In [ ]:
# Solve four ODE
def dALLdt(X,t):
    V, m, h, n = X
    dVdt = (I_inj(t)-I_Na(V, m, h)-I_K(V, n) - I_L(V))/Cm
    dndt = alpha_n(V)*(1-n) - beta_n(V)*n
    dmdt = alpha_m(V)*(1-m) - beta_m(V)*m
    dhdt = alpha_h(V)*(1-h) - beta_h(V)*h
    return dVdt, dmdt, dhdt, dndt
    
X = odeint(dALLdt, [-65, 0.05, 0.6, 0.32], t)
V = X[:,0]
m = X[:,1]
h = X[:,2]
n = X[:,3]
ina = I_Na(V,m,h)
ik = I_K(V, n)
il = I_L(V)
gna = g_NaV(V, m, h)
gk = g_KV(V, n)
gl = g_LV(V)

In [ ]:
plt.figure()

plt.figure(figsize=(20,30))
plt.subplot(5,1,1)
plt.title('Hodgkin-Huxley model simulation')
plt.plot(t, V, 'k')
plt.ylabel('V (mV)')

plt.subplot(5,1,2)
plt.plot(t, ina, 'firebrick', label='$I_{Na}$')
plt.plot(t, ik, 'orange', label = '$I_{K}$')
plt.plot(t, il, 'darkviolet', label='$I_{L}$')
plt.ylabel('Current ($mA/cm^2$)')
plt.legend()

plt.subplot(5,1,3)
plt.plot(t, gna, 'sienna', label='$g_{Na}$')
plt.plot(t, gk, 'salmon', label = '$g_{K}$')
plt.ylabel('Conductance ($mS/cm^2$)')
plt.legend()

plt.subplot(5,1,4)
plt.plot(t, m, 'g', label='m')
plt.plot(t, h, 'r', label='h')
plt.plot(t, n, 'b', label='n')
plt.ylabel('Gating Value')
plt.legend()

plt.subplot(5,1,5)
plt.plot(t, I_inj(t), 'k')
plt.xlabel('t(ms)')
plt.ylabel('$I_{inj}$ ($\\mu{A}/cm^2$)')

plt.show()